# Object GUI

## Choose the working directory

In [3]:
from ipyfilechooser import FileChooser
import os

# Create new FileChooser:
# Path: current directory
# Show hidden files: no
# Use the default path and filename as selection: yes
# Use folder icons: yes
fdialog = FileChooser(
    os.getcwd(),
    filename='',
    title='<b>Change Directory</b>',
    show_hidden=False,
    select_default=True,
    use_dir_icons=True
)

display(fdialog)

# Callback func
def change_directory(chooser):
    os.chdir(chooser.selected)
    print("Change directory to: " + os.getcwd())

# Register callback function
fdialog.register_callback(change_directory)


FileChooser(path='/Users/chris/Desktop', filename='', show_hidden='False')

## Choose Directory

In [6]:
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display
import glob

fdialog2 = FileChooser(
    os.getcwd(),
    filename='',
    title='',
    change_desc = 'Choose',
    show_hidden=False,
    select_default=True,
    use_dir_icons=True
)

display(fdialog2)

module_list = {}
module_sel = widgets.Select()
module_sel.options = module_list.keys()
display(module_sel)

FileChooser(path='/Users/chris/Desktop', filename='', show_hidden='False')

Select(options=(), value=None)

## Object

In [5]:
# create initial widget
object_sel = widgets.Select()
object_sel.options = []
object_sel.rows=10
display(object_sel)

# Callback func
def search_module(chooser):
    cw = os.getcwd()
    os.chdir(chooser.selected)
    module_name = os.path.basename(os.path.normpath(chooser.selected))
    global module_list
    class_list = []
    # look for Module name folder
    if bool(glob.glob(module_name)):
        os.chdir(module_name)
        if bool(glob.glob("__init__.py")):
            ini_file = open("__init__.py")
            for line in ini_file.readlines():
                # skip empty lines
                line = line.strip()
                if not line:  # line is blank
                    continue
                last_word = line.split()[-1]
                if(last_word[0].isupper()):
                    class_list += [last_word]
            ini_file.close()
    # change possible selections
    module_list[module_name] = class_list
    module_sel.options = module_list.keys()
    module_sel.value = module_name
    display(module_sel)
    os.chdir(cw)

# Register callback function
fdialog2.register_callback(search_module)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        module_name = change['new']
#         object_sel.value = None
        object_sel.options = module_list[module_name]
        display(object_sel)

module_sel.observe(on_change)

Select(options=(), value=None)

## Constructor Options

In [12]:
import importlib
from ipywidgets import interact

text_display = widgets.VBox([])
display(text_display)

def on_select_object(change):
    if change['type'] == 'change' and change['name'] == 'value':
        object_name = change['new']
        module_import = importlib.import_module(module_sel.value)
        object_import = getattr(module_import, object_name)
        argsList = getattr(object_import, 'argsList', 'None')
        if argsList != 'None':
            text_list = []
            arg_dict = dict(argsList)
            for key in arg_dict:
                a = widgets.Text(value = str(arg_dict[key]),
                             placeholder='Type something',
                             description= str(key),
                             disabled=False)
                text_list += [a]
            text_display.children=tuple(text_list)     

object_sel.observe(on_select_object)

VBox()